In [ ]:
# big shout out to these folks
# https://github.com/alsnhll/SEIR_COVID19/blob/master/SEIR_COVID19.ipynb

In [ ]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

from scipy.integrate import odeint
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
timeseries_df = pd.read_csv('../data/timeseries.csv')

In [ ]:
beds = pd.read_csv('../data/beds.csv')

In [ ]:
beds.head()

In [ ]:
us_df = timeseries_df.loc[(timeseries_df['country'] == 'USA'), :]

In [ ]:
#us_df.loc[(timeseries_df['state'] == 'TX') & (timeseries_df['cases'] > 0), :]

In [ ]:
us_df['datetime'] = pd.to_datetime(us_df['date'])

# us_df.set_index(['datetime', 'state'], inplace=True)

In [ ]:
print(us_df.shape)
us_last_df = us_df.sort_values('datetime').groupby(['state', 'county']).last().reset_index()
print(us_last_df.shape)

In [ ]:
us_last_df.head()

In [ ]:
us_last_df.loc[:, ['cases', 'deaths', 'recovered', 'active']] = us_last_df.loc[:, ['cases', 'deaths', 'recovered', 'active']].fillna(0)

In [ ]:
# The SIR model differential equations.
def deriv(y, t, N, beta, gamma):
    t = None
    
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

In [ ]:
def sir(start_date, end_date, pop_dict, beta, gamma):
    # Initial conditions vector
    # print(pop_dict['total'], pop_dict['infected'], pop_dict['recovered'], pop_dict['deaths'])
    
    susceptible = pop_dict['total'] - pop_dict['infected'] - pop_dict['recovered'] - pop_dict['deaths']
    
    y0 = susceptible, pop_dict['infected'], pop_dict['recovered']
    
    delta = end_date - start_date
        
    t = np.linspace(0, delta.days, delta.days )
    
    # print(y0, t, beta, gamma)
    ret = odeint(deriv, y0, t, args=(pop_dict['total'], beta,gamma,))
    
    return ret.T

In [ ]:
# https://github.com/alsnhll/SEIR_COVID19/blob/master/SEIR_COVID19.ipynb

#Defining the differential equations

#Don't track S because all variables must add up to 1 
#include blank first entry in vector for beta, gamma, p so that indices align in equations and code. 
#In the future could include recovery or infection from the exposed class (asymptomatics)

def deriv_seird(y,t,N,b,a,g,p,u): 
    dy=[0,0,0,0,0,0]
    S=N-sum(y);
    dy[0]=np.dot(b[1:3],y[1:3])*S-a*y[0] # E
    dy[1]= a*y[0]-(g[1]+p[1])*y[1] #I1
    dy[2]= p[1]*y[1] -(g[2]+p[2])*y[2] #I2
    dy[3]= p[2]*y[2] -(g[3]+u)*y[3] #I3
    dy[4]= np.dot(g[1:3],y[1:3]) #R
    dy[5]=u*y[3] #D

    return dy

In [ ]:
def seird(start_date, end_date, pop_dict, beta, alpha, gamma, rho, mu):
    # Initial conditions vector
    # print(pop_dict['total'], pop_dict['infected'], pop_dict['recovered'], pop_dict['deaths'])
    
    susceptible = pop_dict['total'] - pop_dict['infected'] - pop_dict['recovered'] - pop_dict['deaths']
    
    y0 = susceptible, pop_dict['infected'], pop_dict['recovered'], 0,0,0
    
    delta = end_date - start_date
    
    print(delta)
    
    t = np.linspace(0, delta.days, delta.days )
    
    # print(y0, t, beta, gamma)
    ret = odeint(deriv_seird, y0, t, args=(pop_dict['total'], beta,  alpha, gamma, rho, mu))
    
    return ret.T

In [ ]:
# Define parameters based on clinical observations

#I will add sources soon
# https://github.com/midas-network/COVID-19/tree/master/parameter_estimates/2019_novel_coronavirus



b=np.zeros(4) #beta
g=np.zeros(4) #gamma
p=np.zeros(3)

a=1/IncubPeriod

u=(1/TimeICUDeath)*(CFR/FracCritical)
g[3]=(1/TimeICUDeath)-u

p[2]=(1/DurHosp)*(FracCritical/(FracCritical+FracSevere))
g[2]=(1/DurHosp)-p[2]

g[1]=(1/DurMildInf)*FracMild
p[1]=(1/DurMildInf)-g[1]

#b=2e-4*np.ones(4) # all stages transmit equally
b=2.5e-4*np.array([0,1,0,0]) # hospitalized cases don't transmit

tmax=120
tvec=np.arange(0,tmax,0.1)
ic=np.zeros(6)
ic[0]=1

In [ ]:
end_date = datetime(2020, 12, 31)
beta = 0.2
gamma = 1./10

def run_row(row):

    pop_dict = {
        'total': row['population'],
        'infected': row['active'],
        'recovered': row['recovered'],
        'deaths': row['deaths']
    }
    
    # print(row)
    
    N = pop_dict['total']
    
    #Calculate basic reproductive ratio
    R0=N*((b[1]/(p[1]+g[1]))+(p[1]/(p[1]+g[1]))*(b[2]/(p[2]+g[2])+ (p[2]/(p[2]+g[2]))*(b[3]/(u+g[3]))))
    print("R0 = {0:4.1f}".format(R0))
    
    #(S, I, R) = sir(row['datetime'], end_date, pop_dict, beta, gamma)
    (S, E, Ia, Ib, Ic, R) = seird(row['datetime'], end_date, pop_dict, b, a, g, p, u)
    
    dates = pd.date_range(start=row['datetime'], end=(end_date - timedelta(1)), freq='D').to_list()
    
    state = [row['state']] * len(S)
    county = [row['county']] * len(S)
    
    sir_df = pd.DataFrame(zip(state, county, S, E, Ia, Ib, Ic, R), columns=['state', 'county', 'susceptible', 'exposed',
                                                                            'infectedA', 'infectedB', 'infectedC',
                                                                            'recovered'], index=dates)
    
    return sir_df

In [ ]:
df_list = us_last_df.loc[[0, 5], :].apply(run_row, axis=1)

In [ ]:
len(df_list)

In [ ]:
df_list[0].head()

In [ ]:
def plot_sir(df):
    print(df.head())
    
    df.index.name = 'date'
    df.drop(['state', 'county'], axis=1, inplace=True)

    df = df.reset_index().melt('date', var_name='population', value_name='people')
        
    sir_plot = sns.factorplot(x='date', y='people', hue='population', data=df)
    
    return sir_plot

plt.figure(figsize=(15,8))

plot_sir(df_list[0])

In [ ]:
plot_sir(df_list[183])